# 代码生成器

需求：使用前沿模型将 Python 代码生成为高性能 C++ 代码


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">提醒：执行 C++ 或 Rust 代码是可选的</h2>
            <span style="color:#f71;">作为替代方案，你可以在昨天提供的网站上运行它</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">重要提示</h1>
            <span style="color:#900;">
            在本实验中，我使用高端模型 GPT 5、Claude 4.5 Sonnet、Gemini 2.5 Pro、Grok 4，这些是价格稍高的模型。成本仍然很低，但如果您想将成本保持在极低水平，请选择 gpt-5-nano 等低成本模型。
            </span>
        </td>
    </tr>
</table>

In [2]:
# 导入库

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
siliconcloud_api_key = os.getenv('SILICONCLOUD_API_KEY')

if openai_api_key:
    print(f"OpenAI API 密钥存在，开头为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
if anthropic_api_key:
    print(f"Anthropic API 密钥存在，开头为 {anthropic_api_key[:7]}")
else:
    print("未设置 Anthropic API 密钥（此项为可选）")

if google_api_key:
    print(f"Google API 密钥存在，开头为 {google_api_key[:2]}")
else:
    print("未设置 Google API 密钥（此项为可选）")

if grok_api_key:
    print(f"Grok API 密钥存在，开头为 {grok_api_key[:4]}")
else:
    print("未设置 Grok API 密钥（此项为可选）")

if groq_api_key:
    print(f"Groq API 密钥存在，开头为 {groq_api_key[:4]}")
else:
    print("未设置 Groq API 密钥（此项为可选）")

if openrouter_api_key:
    print(f"OpenRouter API 密钥存在，开头为 {openrouter_api_key[:6]}")
else:
    print("未设置 OpenRouter API 密钥（此项为可选）")

if siliconcloud_api_key:
    print(f"siliconcloud API 密钥存在，开头为 {siliconcloud_api_key[:4]}")
else:
    print("未设置 siliconcloud API 密钥（此项为可选）")

OpenAI API 密钥存在，开头为 sk-proj-
Anthropic API 密钥存在，开头为 sk-ant-
Google API 密钥存在，开头为 AI
Grok API 密钥存在，开头为 xai-
Groq API 密钥存在，开头为 gsk_
未设置 OpenRouter API 密钥（此项为可选）
siliconcloud API 密钥存在，开头为 sk-t


In [4]:
# 连接到客户端库

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"
siliconcloud_url = "https://api.siliconflow.cn/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
siliconcloud = OpenAI(api_key=siliconcloud_api_key, base_url=siliconcloud_url)

In [5]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "gemini-2.5-pro", "openai/gpt-oss-120b", "deepseek-ai/DeepSeek-V3.2-Exp", "openai/gpt-oss-20b", "Qwen/Qwen3-Coder-30B-A3B-Instruct", "qwen2.5-coder"]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "deepseek-ai/DeepSeek-V3.2-Exp": siliconcloud, "openai/gpt-oss-20b": groq, "Qwen/Qwen3-Coder-30B-A3B-Instruct": siliconcloud, "qwen2.5-coder": ollama}

In [6]:
from system_info import retrieve_system_info, rust_toolchain_info

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
rust_info

{'installed': True,
 'rustc': {'path': '/opt/homebrew/bin/rustc',
  'version': 'rustc 1.91.1 (ed61e7d7e 2025-11-07) (Homebrew)',
  'host_triple': 'aarch64-apple-darwin',
  'release': '1.91.1',
  'commit_hash': 'ed61e7d7e242494fb7057f2657300d9e77bb4fcb'},
 'cargo': {'path': '/opt/homebrew/bin/cargo',
  'version': 'cargo 1.91.1 (Homebrew)'},
 'rustup': {'path': '',
  'version': '',
  'active_toolchain': '',
  'default_toolchain': '',
  'toolchains': [],
  'targets_installed': []},
 'rust_analyzer': {'path': ''},
 'env': {'CARGO_HOME': '',
  'RUSTUP_HOME': '',
  'RUSTFLAGS': '',
  'CARGO_BUILD_TARGET': ''},
 'execution_examples': ['"/opt/homebrew/bin/cargo" build',
  '"/opt/homebrew/bin/cargo" run',
  '"/opt/homebrew/bin/cargo" test',
  '"/opt/homebrew/bin/rustc" hello.rs -o hello']}

In [7]:
message = f"""
这是我的计算机系统信息报告。
我想运行 Rust 编译器编译一个名为 main.rs 的单个 Rust 文件，然后以最简单的方式执行它。
请回复我是否需要安装 Rust 工具链来执行此操作。如果是，请提供最简单的分步说明。

如果我已经设置好可以编译 Rust 代码，那么我想在 Python 中运行类似下面的代码来编译和执行代码：
```python
compile_command = # 这里填写内容 - 以实现最快的运行时性能
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # 这里填写内容
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
请确切告诉我 compile_command 和 run_command 应该用什么。
请牢记最大可能的运行时性能；编译时间可以慢。此平台的尽可能最快运行时性能是关键。
请用 markdown 回复命令。

系统信息：
{system_info}

Rust 工具链信息：
{rust_info}
"""

response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

不需要安装。你的系统已安装 Rust 工具链：
- rustc: /opt/homebrew/bin/rustc (1.91.1)
- cargo: /opt/homebrew/bin/cargo (1.91.1)

最简单的一次性编译并运行单文件
```sh
/opt/homebrew/bin/rustc main.rs -O -o main
./main
```

Python 中为“尽可能最快运行时性能”的命令（编译可慢）
```python
compile_command = [
    "/opt/homebrew/bin/rustc",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "lto=fat",
    "-C", "codegen-units=1",
    "-C", "panic=abort",
    "main.rs",
    "-o", "main",
]

run_command = ["./main"]
```

说明：
- 以上编译参数偏向极致运行时性能（牺牲编译时间）：O3、LTO(fat)、单代码生成单元、panic 直接 abort、按本机 CPU 指令集优化。
- 如遇链接过慢或内存压力大，可将 lto=fat 改为 lto=thin。

## 对于 C++，请用昨天的命令覆盖此处，或者对于 Rust，使用新命令

或者直接像昨天一样使用网站：

 https://www.programiz.com/cpp-programming/online-compiler/

In [8]:
compile_command = [
    "/opt/homebrew/bin/rustc", "main.rs",
    "-C", "opt-level=3",
    "-C", "lto=fat",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "panic=abort",
    "-C", "strip=symbols",
    "-o", "main",
]
run_command = ["./main"]



## 现在，继续主要任务

In [9]:
language = "Rust" # 或 "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
你的任务是将 Python 代码转换为高性能 {language} 代码。
仅回复 {language} 代码。除偶尔的注释外，不要提供任何解释。
{language} 响应需要产生相同的输出，并以尽可能快的时间运行。
"""

def user_prompt_for(python):
    return f"""
将此 Python 代码移植到 {language}，采用产生相同输出且耗时最少的最快实现。
系统信息如下：
{system_info}
你的响应将被写入名为 main.{language} 的文件，然后编译并执行；编译命令是：
{compile_command}
仅回复 {language} 代码。
要移植的 Python 代码：

```python
{python}
```
"""

In [10]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [11]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [12]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```rust','').replace('```','')
    return reply

In [13]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [14]:
# 使用 GPT 5 的命令

def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"发生错误：\n{e.stderr}"

In [15]:
python_hard = """# 注意支持大数

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# 参数
n = 10000         # 随机数数量
initial_seed = 42 # LCG 的初始种子
min_val = -10     # 随机数最小值
max_val = 10      # 随机数最大值

# 函数计时
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("最大子数组总和（20次运行）：", result)
print("执行时间：{:.6f} 秒".format(end_time - start_time))
"""

In [16]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"从 Python 移植到 {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (原始)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (已生成)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("运行 Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"移植到 {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"运行 {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python 结果", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} 结果", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 结果！

Qwen 2.5 Coder:失败 
Gemini 2.5 Pro:  失败 
DeepSeek Coder v2:   
Qwen3 Coder 30B: 失败 
Claude Sonnet 4.5: 0.510052 秒 
GPT-5:   失败

GPT-oss-20B: 0.000441 秒
DeepSeek Coder v3.2-exp ：0.517202 秒
OpenAI GPT-OSS 120B: 0.000579 秒 

In [17]:
print(f"在 沉默恶魔 的实验中，GPT-OSS 120B 模型的结果比 Python 代码快 {16.158406/0.000441:,.0f} 倍。")

在 沉默恶魔 的实验中，GPT-OSS 120B 模型的结果比 Python 代码快 36,640 倍。
